In [ ]:
from qdrant_client import QdrantClient
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()

In [3]:
client = QdrantClient(path="./qdrant_storage/")
embeddings = OpenAIEmbeddings()

In [5]:
collection = "toronto_sf"

In [13]:
qdrant=Qdrant(client=client, collection_name=collection, embeddings=embeddings)

In [14]:
client.get_collection(collection_name=collection)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=258, indexed_vectors_count=0, points_count=258, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1536, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [15]:
qdrant.similarity_search("population of San Francisco", k=1)

[Document(page_content="The 2020 United States census showed San Francisco's population to be 873,965, an increase of 8.5% from the 2010 census. With roughly one-quarter the population density of Manhattan, San Francisco is the second-most densely populated large American city, behind only New York City among cities greater than 200,000 population, and the fifth-most densely populated U.S. county, following only four of the five New York City boroughs.  \nSan Francisco is part of the five-county San Francisco–Oakland–Hayward, CA Metropolitan Statistical Area, a region of 4.7\xa0million people (13th most populous in the U.S.), and has served as its traditional demographic focal point. It is also part of the greater 14-county San Jose-San Francisco-Oakland, CA Combined Statistical Area, whose population is over 9.6\xa0million, making it the fifth-largest in the United States as of 2018.", metadata={'Header 2': 'Demographics'})]

In [16]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter

In [18]:
with open('../langchain/lcel/custom_embedding/data/Houston.txt', 'r') as f:
    html_toronto = f.read()
with open('../langchain/lcel/custom_embedding/data/New_York_City.txt', 'r') as f:
    html_sf = f.read()
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
houston_header_splits = html_splitter.split_text(html_toronto)
nyc_header_splits = html_splitter.split_text(html_sf)

houston_header_splits[0].metadata = {"Header2": "Overview", "Header 3": "population"}
nyc_header_splits[0].metadata = {"Header2": "Overview", "Header 3": "population"}
total_headers = houston_header_splits + nyc_header_splits
len(total_headers)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=80)
texts = text_splitter.split_documents(total_headers)
len(texts)

304

In [19]:
qdrant.add_documents(texts)

['a3d96abfec07499b995673600a31ed4c',
 'e14ae8f38c5249c18a0109444da82b8e',
 '33fcea877ad24341b39c3dbeae4f4951',
 '34536d9b86c2456e9e360524a87c7906',
 '4bd20c8e9de24432b877f459758f1426',
 '8e506012b53a430b84500d637c84fa77',
 'f8384c058862422c928b2d4f08e4a580',
 '246a57c37df1402f9e2e2f96f20cfe63',
 'a0bb5e13f48f4e6eaf014649e6f1c88e',
 '22428c3fa0bf48eaaa70f9d4571b4898',
 '9b40f797783a465fa8ae652a0ac269d8',
 '3ee7d77317f741b3b3145d737f61d6e1',
 'edd9acdfeed2486cbc4d7c0ec66419ca',
 'd58b956709044c95a56919e83fe64ee7',
 '7f9e89ff700a4c5581d0af93e860608b',
 '7754b48bbee6485d9faa62e17b8af29b',
 'cf82809af54d4a1889c3f277ac83e30e',
 'aaf7d3a6bfef4a279e0f24546de58032',
 'b02d97805dd144ee9f749e8db338fb5f',
 'a727be2853d34691997c3f2ee5e381d7',
 '2bec236cb516420fbe41e28eee05f9df',
 '32e9514a19ef47fbabf11e892c44c9b3',
 '28d739227fd74ca0b0186dcbc5e3647c',
 '9af18edaf0a84e1d9a61cb08d4f25375',
 '67d914d2a8ed408dae8689cd89d37391',
 'f1367114facf40cb97c7d6bc1909cc78',
 'af9ae7a729284fbfa3a72e83928a490f',
 

In [20]:
qdrant.similarity_search("population of Houston", k=1)

[Document(page_content="The 2020 U.S. census determined Houston had a population of 2,304,580. In 2017, the census-estimated population was 2,312,717, and in 2018 it was 2,325,502. An estimated 600,000 undocumented immigrants resided in the Houston area in 2017, comprising nearly 9% of the city's metropolitan population. At the 2010 United States census, Houston had a population of 2,100,263 residents, up from the city's 2,396 at the 1850 census.", metadata={'Header 2': 'Demographics'})]

In [23]:
print(texts[0].metadata)

{'Header2': 'Overview', 'Header 3': 'population'}


In [24]:
client.count(collection_name=collection)

CountResult(count=562)

In [31]:
client.scroll(collection_name=collection)

([Record(id='00a150acc1fb496691755ab5ea75b8b1', payload={'page_content': 'Broadway theatre is one of the premier forms of English-language theatre in the world, named after Broadway, the major thoroughfare that crosses Times Square, sometimes referred to as "The Great White Way". Forty-one venues in Midtown Manhattan\'s Theatre District, each with at least 500 seats, are classified as Broadway theatres. According to The Broadway League, Broadway shows sold approximately $1.27\xa0billion worth of tickets in the 2013–2014 season, an 11.4% increase from $1.139\xa0billion in the 2012–2013 season. Attendance in 2013–2014 stood at 12.21\xa0million, representing a 5.5% increase from the 2012–2013 season\'s 11.57\xa0million. Performance artists displaying diverse skills are ubiquitous on the streets of Manhattan.', 'metadata': {'Header 2': 'Culture', 'Header 3': 'Arts'}}, vector=None),
  Record(id='00ef8ffa519e43a79272ab93887a8353', payload={'page_content': 'Some of the most popular tourist at

In [32]:
new_vector = OpenAIEmbeddings().embed_query("this is a test")
print(new_vector)

[-0.01286368747985042, -0.007444602110703561, -0.01758546415034265, -0.028397072636989644, -0.01870115881324813, 0.01741279694178393, -0.01822300475599372, -0.006966447122126556, -0.009934990852368718, -0.0386508337921839, 0.01073855647385229, 0.024093680533764376, -0.013574278235651006, 0.013209021896967866, 0.002090266246274278, 0.01235232804690047, 0.02078644411346969, 0.0007599836240908414, 0.008380987837985396, -0.005282944632646784, 0.00906501447949407, 0.014344638481439041, -0.014118843975619086, 0.01503530638435134, -0.022247473193492618, -0.02515624696808603, 0.006465049115620748, -0.03639287942075843, 0.009244322949456419, -0.009709195415226171, 0.009443554272307053, -0.00843411606656922, -0.004064314143275469, -0.034453698146126276, -0.0009040110913383712, -0.001952464728761999, 0.002282856307721286, -0.021530241176288406, 0.0164299237816606, -0.015553308010027508, 0.0038551216268113394, -6.324549876701557e-05, 0.0032441462996054236, -0.0203614182847991, -0.00968263130093425

In [33]:
from qdrant_client import models
#let's update one Point manually with the embedding vector
client.upsert(
    collection_name="toronto_sf",
    points=[
        models.PointStruct(
            id=1,
            payload={
                "test": "adding a new point manually",
            },
            vector=new_vector,
        ),
    ],
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [34]:
client.retrieve(collection_name=collection, ids=[1])

[Record(id=1, payload={'test': 'adding a new point manually'}, vector=None)]